In [2]:
import utils
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from scipy.sparse import hstack
import pandas as pd
from skrub import TableReport
from jours_feries_france import JoursFeries
from vacances_scolaires_france import SchoolHolidayDates
import numpy as np

In [ ]:
X, y, X_test = utils.get_and_process_data()

/Users/felix/Python_Data_Challenge-1/utils.py:48: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
  closest_value = non_nan_values.iloc[(non_nan_values - value).abs().argmin()]
/Users/felix/Python_Data_Challenge-1/utils.py:48: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
  closest_value = non_nan_values.iloc[(non_nan_values - value).abs().argmin()]
/Users/felix/Python_Data_Challenge-1/utils.py:197: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting val

In [ ]:
import pandas as pd
from flaml import AutoML
from skrub import TableVectorizer


# Split the data into training and validation sets based on the last 10% of dates
validation_split_index = int(len(X) * 0.9)
X_train, X_val = X.iloc[:validation_split_index], X.iloc[validation_split_index:]
y_train, y_val = y.iloc[:validation_split_index], y.iloc[validation_split_index:]

# Initialize the TableVectorizer
vectorizer = TableVectorizer()

# Fit and transform the training data
X_train_transformed = vectorizer.fit_transform(X_train)
X_val_transformed = vectorizer.transform(X_val)

In [ ]:
import optuna
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import root_mean_squared_error
import joblib

# Sanitize feature names
def sanitize_feature_names(df):
    """Sanitize feature names to remove special JSON characters."""
    df.columns = df.columns.str.replace(r"[^\w\d_]", "_", regex=True)
    return df

# Sanitize training and validation feature names
X_train_transformed = sanitize_feature_names(pd.DataFrame(X_train_transformed))
X_val_transformed = sanitize_feature_names(pd.DataFrame(X_val_transformed))

# Define the objective function for Optuna
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 200),  # Number of trees
        'max_depth': trial.suggest_int('max_depth', 2, 20),         # Maximum depth of trees
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),  # Min samples to split
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),    # Min samples per leaf
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),  # Max features
    }
    model = RandomForestRegressor(**param, random_state=42, n_jobs=-1)
    model.fit(X_train_transformed, y_train)
    y_pred = model.predict(X_val_transformed)
    rmse = root_mean_squared_error(y_val, y_pred, squared=False)
    return rmse  # Minimize RMSE

# Create a study and optimize the objective function
study = optuna.create_study(direction='minimize')  # Minimizing RMSE
study.optimize(objective, n_trials=50)

# Get the best parameters
best_params = study.best_params

# Save the best parameters to a file using joblib
joblib.dump(best_params, 'random_forest_best_params.pkl')

# Train the final model with the best parameters
X_transformed = sanitize_feature_names(pd.DataFrame(vectorizer.transform(X)))  # Transform the entire dataset
final_model = RandomForestRegressor(**best_params, random_state=42, n_jobs=-1)
final_model.fit(X_transformed, y)
